# Making a Dataset of "Normal" Election Tweets

In [315]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import json
import requests
#require('dotenv').config()

load_dotenv()
def auth():
    return os.getenv('MY_BEARER')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
#keyword = "xbox lang:en"
#start_time = "2021-11-02T00:00:00.000Z"
#end_time = "2021-11-04T00:00:00.000Z"
#max_results = 10

In [4]:
filter1 = np.loadtxt(fname='election-filter1.txt', dtype=int)
filter1 #This is a LOT of id #s

array([763431264116416512, 763431263545896960, 763431264577728512, ...,
       782631555839569921, 782631556682772480, 782631556313518080])

In [6]:
normal_id = pd.DataFrame(filter1, columns = ['tweet_id'])
normal_id

,tweet_id
0,763431264116416512
1,763431263545896960
2,763431264577728512
3,763431264061755392
4,763431265361993728
...,...
49999995,782631556359782403
49999996,782631555952816128
49999997,782631555839569921
49999998,782631556682772480


In [7]:
normal_id.to_csv('normal_ids.csv')

In [24]:
for id in normal_id[:2000]['tweet_id']:
    print(id)

763431264116416512
763431263545896960
763431264577728512
763431264061755392
763431265361993728
763431265177595905
763431265370505216
763431265660002304
763431265252945921
763431265743802368
763431265508859908
763431265701801984
763431266171645952
763431266158911488
763431265479516160
763431266347782145
763431264393322496
763431265437487104
763431266150526976
763431266918232065
763431267304140801
763431267362672640
763431267471880192
763431267744505856
763431267668938752
763431267501125632
763431267094454273
763431268289683457
763431268629479424
763431268587626496
763431268558200833
763431268914724864
763431268960837632
763431268402987008
763431267929038848
763431268935622656
763431269233491968
763431269376098304
763431270022012930
763431270063931392
763431270433030144
763431270089158657
763431270391091202
763431270005092352
763431269917036544
763431270013558784
763431268709175296
763431270877650944
763431270621716480
763431271045488640
763431271301312512
763431271104118784
763431271343

In [3]:
#Right now, below, you have to hard-code the tweet id inside the url, but this can be fixed. Need to import
#the necessary tweet ids from the Harvard study to run through them all and output needed data to a csv.
#First: MAKE SURE YOU'RE PULLING EVERYTHING YOU NEED IN THE PARAMETERS
#for id in normal_id[:2000]['tweet_id']:
def create_url():
    #keyword, start_date, end_date, max_results = 10 #goes into the above

    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
    search_url = 'https://api.twitter.com/1.1/statuses/show.json?id=763431266171645952'
    #change params based on the endpoint you are using
    query_params = {#'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    #'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

url = create_url()

#keyword, start_time, end_time, max_results
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

json_response = connect_to_endpoint(url[0], headers, url[1])
#url = create_url(keyword, start_time,end_time, max_results)
#json_response = connect_to_endpoint(url[0], headers, url[1])

print(json.dumps(json_response, indent=4, sort_keys=True))

Endpoint Response Code: 200
{
    "contributors": null,
    "coordinates": null,
    "created_at": "Wed Aug 10 17:46:12 +0000 2016",
    "entities": {
        "hashtags": [],
        "symbols": [],
        "urls": [],
        "user_mentions": [
            {
                "id": 51433856,
                "id_str": "51433856",
                "indices": [
                    3,
                    15
                ],
                "name": "Dave Manchester",
                "screen_name": "dredeyedick"
            }
        ]
    },
    "favorite_count": 0,
    "favorited": false,
    "geo": null,
    "id": 763431266171645952,
    "id_str": "763431266171645952",
    "in_reply_to_screen_name": null,
    "in_reply_to_status_id": null,
    "in_reply_to_status_id_str": null,
    "in_reply_to_user_id": null,
    "in_reply_to_user_id_str": null,
    "is_quote_status": false,
    "lang": "en",
    "place": null,
    "retweet_count": 2,
    "retweeted": false,
    "retweeted_status": {
    

In [11]:
print(json_response)

{'created_at': 'Wed Aug 10 17:46:12 +0000 2016', 'id': 763431266171645952, 'id_str': '763431266171645952', 'text': 'RT @dredeyedick: If Trump was "joking", why say "&amp; that will be a horrible day."?This was no joke. It was fomenting Insurrection. https://t…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'dredeyedick', 'name': 'Dave Manchester', 'id': 51433856, 'id_str': '51433856', 'indices': [3, 15]}], 'urls': []}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 288080988, 'id_str': '288080988', 'name': 'mary hendrix', 'screen_name': 'hendry118', 'location': '', 'description': 'Retired Fed.  Remember McCarthy, Nixon eras.  Alarmed about govt. surveillance.', 'url': None, 'entities': {'description': {'urls': []

In [29]:
t_user= json_response['user']
#don't need to push the above, just for below
t_id = json_response['id']
t_text = json_response['text']
t_created_at = json_response['created_at']
t_hashtags = json_response['entities']['hashtags']
t_mentions = json_response['entities']['user_mentions']
t_urls = json_response['entities']['urls']
t_source = json_response['source']
t_lang = json_response['lang']
t_favorite_count = json_response['favorite_count']
t_retweet_count = json_response['retweet_count']
t_retweeted = json_response['retweeted']
t_retweet_status_id = json_response['retweeted_status']['id']
t_in_reply_to_status_id = json_response['in_reply_to_status_id']
#print("favorite count: ", t_favorite_count, "retweet count: ", t_retweet_count, "retweeted: ", t_retweeted)
#print(t_retweet_status_id)
#print(t_in_reply_to_status_id)

favorite count:  0 retweet count:  2 retweeted:  False
763212860860329984
None


In [19]:
user_id = t_user['id']
user_name = t_user['name']
user_handle = t_user['screen_name']
user_location = t_user['location']
user_desc = t_user['description']

288080988 mary hendrix hendry118  Retired Fed.  Remember McCarthy, Nixon eras.  Alarmed about govt. surveillance.


In [21]:
user_verified = t_user['verified'] 
user_lang = t_user['lang']

{'id': 288080988, 'id_str': '288080988', 'name': 'mary hendrix', 'screen_name': 'hendry118', 'location': '', 'description': 'Retired Fed.  Remember McCarthy, Nixon eras.  Alarmed about govt. surveillance.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 560, 'friends_count': 1917, 'listed_count': 185, 'created_at': 'Tue Apr 26 06:24:07 +0000 2011', 'favourites_count': 27289, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 30694, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/378800000129978888/f650f26afd852b430ce79b11fc601a8d_normal.jpe

In [25]:
#print(t_user)

{'id': 288080988, 'id_str': '288080988', 'name': 'mary hendrix', 'screen_name': 'hendry118', 'location': '', 'description': 'Retired Fed.  Remember McCarthy, Nixon eras.  Alarmed about govt. surveillance.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 560, 'friends_count': 1917, 'listed_count': 185, 'created_at': 'Tue Apr 26 06:24:07 +0000 2011', 'favourites_count': 27289, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 30694, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/378800000129978888/f650f26afd852b430ce79b11fc601a8d_normal.jpe

In [26]:
user_followers_count= t_user['followers_count']
user_status_count = t_user['statuses_count']
user_friend_count = t_user['friends_count']
user_favorites_count = t_user['favourites_count']
user_listed_count = t_user['listed_count']
user_listed_count = t_user['time_zone']
user_protected = t_user['protected']
user_created_at = t_user['created_at']

In [35]:
list_t_id = []
list_t_text = []
list_t_created_at = []
list_t_hashtags = []
list_t_mentions = []
list_t_urls = []
list_t_source = []
list_t_lang = []
list_t_favorite_count = []
list_t_retweet_count = []
list_t_retweeted = []
list_t_retweet_status_id = []
list_t_in_reply_to_status_id = []
    
list_user_id = []
list_user_name = []
list_user_handle = []
list_user_location = []
list_user_desc = []
list_user_verified = []
list_user_lang = []
list_user_followers_count= []
list_user_status_count = []
list_user_friend_count = []
list_user_favorites_count = []
list_user_listed_count = []
list_user_time_zone = []
list_user_protected = []
list_user_created_at = []

for id in normal_id[0:10]['tweet_id']:
    def create_url(id):
    #keyword, start_date, end_date, max_results = 10 #goes into the above

    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
        search_url = 'https://api.twitter.com/1.1/statuses/show.json?id='+str(id)
    #change params based on the endpoint you are using
        query_params = {#'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    #'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
        return (search_url, query_params)

    url = create_url(id)

#keyword, start_time, end_time, max_results
    def connect_to_endpoint(url, headers, params, next_token = None):
        params['next_token'] = next_token   #params object received from create_url function
        response = requests.request("GET", url, headers = headers, params = params)
        print("Endpoint Response Code: " + str(response.status_code))
        if response.status_code != 200:
           # raise Exception(response.status_code, response.text)
            pass
        return response.json()

    json_response = connect_to_endpoint(url[0], headers, url[1])
    if len(json_response) > 1:
        t_user= json_response['user']
        #don't need to push the above, just for below
        t_id = json_response['id']
        list_t_id.append(t_id)
    
        t_text = json_response['text']
        list_t_text.append(t_text)
    
        t_created_at = json_response['created_at']
        list_t_created_at.append(t_created_at)
    
        t_hashtags = json_response['entities']['hashtags']
        list_t_hashtags.append(t_hashtags)
    
        t_mentions = json_response['entities']['user_mentions']
        list_t_mentions.append(t_mentions)
    
        t_urls = json_response['entities']['urls']
        list_t_urls.append(t_urls)
    
        t_source = json_response['source']
        list_t_source.append(t_source)
    
        t_lang = json_response['lang']
        list_t_lang.append(t_lang)
    
        t_favorite_count = json_response['favorite_count']
        list_t_favorite_count.append(t_favorite_count)
    
        t_retweet_count = json_response['retweet_count']
        list_t_retweet_count.append(t_retweet_count)
    
        t_retweeted = json_response['retweeted']
        list_t_retweeted.append(t_retweeted)
    
        t_retweet_status_id = json_response['retweeted_status']['id']
        list_t_retweet_status_id.append(t_retweet_status_id)
    
        t_in_reply_to_status_id = json_response['in_reply_to_status_id']
        list_t_in_reply_to_status_id.append(t_in_reply_to_status_id)
    
        user_id = t_user['id']
        list_user_id.append(user_id)
    
        user_name = t_user['name']
        list_user_name.append(user_name)
    
        user_handle = t_user['screen_name']
        list_user_handle.append(user_handle)
    
        user_location = t_user['location']
        list_user_location.append(user_location)
    
        user_desc = t_user['description']
        list_user_desc.append(user_desc)
    
        user_verified = t_user['verified'] 
        list_user_verified.append(user_verified)
    
        user_lang = t_user['lang']
        list_user_lang.append(user_lang)
    
        user_followers_count= t_user['followers_count']
        list_user_followers_count.append(user_followers_count)
    
        user_status_count = t_user['statuses_count']
        list_user_status_count.append(user_status_count)
    
        user_friend_count = t_user['friends_count']
        list_user_friend_count.append(user_friend_count)
    
        user_favorites_count = t_user['favourites_count']
        list_user_favorites_count.append(user_favorites_count)
    
        user_listed_count = t_user['listed_count']
        list_user_listed_count.append(user_listed_count)
    
        user_time_zone = t_user['time_zone']
        list_user_time_zone.append(user_time_zone)
    
        user_protected = t_user['protected']
        list_user_protected.append(user_protected)
    
        user_created_at = t_user['created_at']
        list_user_created_at.append(user_created_at)
    else:
        pass

Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200


In [36]:
print(list_user_friend_count)

[2662, 1600, 3736, 1880]


In [37]:
print(list_user_id, list_user_name, list_user_handle, list_user_location, list_user_desc)

[387465176, 20634939, 4795222485, 2340935540] ['Memeditorialist', '👑Sia Ayrom👑🔆🦁', 'The Desk Man', 'William Lee'] ['ajhtweeting', 'siaayrom', 'RealDeskMan', 'wpl1960'] ['New Westminster, BC', 'Houston, Texas', 'Paradise', 'Hammond, Louisiana'] ["Canada's only clinically tested Memeditorialist. Basically a Photoshop doodler. I *will* misspell. Images 99% Photoshopped. They can't all be gems. NDP.", '\u200fLight Shall Defeat Darkness 👑 airyanem xvareno yazamaide فر ایران را میستاییم \n\u200e#KingdomWithPahlavi\n\u200e#نه_به_جمهوری_اسلامی\n#جاویدشاه', 'Live Love Let Live. Protect and Respect your family. Kill If You Have To. If you stop improving, you start failing. Leave a good legacy, life is short. Rejoice.', '']


In [38]:
print(list_t_text)

['RT @jasonfekete: Great read here: In 2007, Trump was forced to own up to his falsehoods. And he did. Sort of. https://t.co/yM6JGjrWbD', "RT @HuffPostBiz: Trump's new economic plan is short on specifics, other than tax help for wealthy https://t.co/vGefdXaiDP https://t.co/Obh9…", 'RT @nytopinion: Forget politics, Trump is a disgusting human being. His children should be ashamed of him. https://t.co/EErtmVZr4R https://…', 'RT @newtgingrich: Hillary Clinton snubs police union https://t.co/5b2xCcWg9l']


In [39]:
print(list_t_mentions)

[[{'screen_name': 'jasonfekete', 'name': 'Jason Fekete', 'id': 17370412, 'id_str': '17370412', 'indices': [3, 15]}], [{'screen_name': 'HuffPostBiz', 'name': 'HuffPost Business', 'id': 27073265, 'id_str': '27073265', 'indices': [3, 15]}], [{'screen_name': 'nytopinion', 'name': 'New York Times Opinion', 'id': 16686144, 'id_str': '16686144', 'indices': [3, 14]}], [{'screen_name': 'newtgingrich', 'name': 'Newt Gingrich', 'id': 20713061, 'id_str': '20713061', 'indices': [3, 16]}]]


In [ ]:
#Seems like it's working to me. And skipping over the ids that no longer work (which seem to be a lot)

In [56]:
list_t_id = []
list_t_text = []
list_t_created_at = []
list_t_hashtags = []
list_t_mentions = []
list_t_urls = []
list_t_source = []
list_t_lang = []
list_t_favorite_count = []
list_t_retweet_count = []
list_t_retweeted = []
list_t_retweet_status_id = []
list_t_in_reply_to_status_id = []
    
list_user_id = []
list_user_name = []
list_user_handle = []
list_user_location = []
list_user_desc = []
list_user_verified = []
list_user_lang = []
list_user_followers_count= []
list_user_status_count = []
list_user_friend_count = []
list_user_favorites_count = []
list_user_listed_count = []
list_user_time_zone = []
list_user_protected = []
list_user_created_at = []

for id in normal_id[0:1000]['tweet_id']:
    def create_url(id):
    #keyword, start_date, end_date, max_results = 10 #goes into the above

    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
        search_url = 'https://api.twitter.com/1.1/statuses/show.json?id='+str(id)
    #change params based on the endpoint you are using
        query_params = {#'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    #'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
        return (search_url, query_params)

    url = create_url(id)

#keyword, start_time, end_time, max_results
    def connect_to_endpoint(url, headers, params, next_token = None):
        params['next_token'] = next_token   #params object received from create_url function
        response = requests.request("GET", url, headers = headers, params = params)
        print("Endpoint Response Code: " + str(response.status_code))
        if response.status_code != 200:
           # raise Exception(response.status_code, response.text)
            pass
        return response.json()

    json_response = connect_to_endpoint(url[0], headers, url[1])
    if len(json_response) > 1:
        t_user= json_response['user']
        #don't need to push the above, just for below
        t_id = json_response['id']
        list_t_id.append(t_id)
    
        t_text = json_response['text']
        list_t_text.append(t_text)
    
        t_created_at = json_response['created_at']
        list_t_created_at.append(t_created_at)
    
        t_hashtags = json_response['entities']['hashtags']
        list_t_hashtags.append(t_hashtags)
    
        t_mentions = json_response['entities']['user_mentions']
        list_t_mentions.append(t_mentions)
    
        t_urls = json_response['entities']['urls']
        list_t_urls.append(t_urls)
    
        t_source = json_response['source']
        list_t_source.append(t_source)
    
        t_lang = json_response['lang']
        list_t_lang.append(t_lang)
    
        t_favorite_count = json_response['favorite_count']
        list_t_favorite_count.append(t_favorite_count)
    
        t_retweet_count = json_response['retweet_count']
        list_t_retweet_count.append(t_retweet_count)
    
        t_retweeted = json_response['retweeted']
        list_t_retweeted.append(t_retweeted)
        
        if 'retweeted_status' in json_response:
            t_retweet_status_id = json_response['retweeted_status']['id']
            list_t_retweet_status_id.append(t_retweet_status_id)
        else:
            list_t_retweet_status_id.append("None")
    
        t_in_reply_to_status_id = json_response['in_reply_to_status_id']
        list_t_in_reply_to_status_id.append(t_in_reply_to_status_id)
    
        user_id = t_user['id']
        list_user_id.append(user_id)
    
        user_name = t_user['name']
        list_user_name.append(user_name)
    
        user_handle = t_user['screen_name']
        list_user_handle.append(user_handle)
    
        user_location = t_user['location']
        list_user_location.append(user_location)
    
        user_desc = t_user['description']
        list_user_desc.append(user_desc)
    
        user_verified = t_user['verified'] 
        list_user_verified.append(user_verified)
    
        user_lang = t_user['lang']
        list_user_lang.append(user_lang)
    
        user_followers_count= t_user['followers_count']
        list_user_followers_count.append(user_followers_count)
    
        user_status_count = t_user['statuses_count']
        list_user_status_count.append(user_status_count)
    
        user_friend_count = t_user['friends_count']
        list_user_friend_count.append(user_friend_count)
    
        user_favorites_count = t_user['favourites_count']
        list_user_favorites_count.append(user_favorites_count)
    
        user_listed_count = t_user['listed_count']
        list_user_listed_count.append(user_listed_count)
    
        user_time_zone = t_user['time_zone']
        list_user_time_zone.append(user_time_zone)
    
        user_protected = t_user['protected']
        list_user_protected.append(user_protected)
    
        user_created_at = t_user['created_at']
        list_user_created_at.append(user_created_at)
    else:
        pass

Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Co

Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Co

KeyboardInterrupt: 

In [57]:
print(len(list_t_retweet_status_id))

100


In [58]:
print(len(list_t_text))

100


In [59]:
print(len(list_user_lang))

100


In [60]:
#Hit 429 which apparently means I'm making too many requests and only wound up with 100 completed responses. Great.
#It's messed up how many weren't even accessible/existing accounts but I don't know of a way to check that before running
#the request.

In [61]:
completed_dict = {}
completed_dict['t_id'] = list_t_id
completed_dict['t_text'] = list_t_text
completed_dict['t_created_at'] = list_t_created_at
completed_dict['t_hashtags'] = list_t_hashtags
completed_dict['t_mentions'] = list_t_mentions
completed_dict['t_urls'] = list_t_urls
completed_dict['t_source'] = list_t_source
completed_dict['t_lang'] = list_t_lang
completed_dict['t_favorite_count'] = list_t_favorite_count
completed_dict['t_retweet_count'] = list_t_retweet_count
completed_dict['t_retweeted'] = list_t_retweeted
completed_dict['t_retweet_status_id'] = list_t_retweet_status_id
completed_dict['t_in_reply_to_status_id'] = list_t_in_reply_to_status_id
    
completed_dict['user_id'] = list_user_id
completed_dict['user_name'] = list_user_name
completed_dict['user_handle'] = list_user_handle
completed_dict['user_locatoin'] = list_user_location
completed_dict['user_desc'] = list_user_desc
completed_dict['user_verified'] = list_user_verified
completed_dict['user_lang'] = list_user_lang
completed_dict['followers_count'] = list_user_followers_count
completed_dict['user_status_count'] = list_user_status_count
completed_dict['user_friend_count'] = list_user_friend_count
completed_dict['user_favorites_count'] = list_user_favorites_count
completed_dict['user_listed_count'] = list_user_listed_count
completed_dict['user_time_zone'] = list_user_time_zone
completed_dict['user_protected'] = list_user_protected
completed_dict['user_created_at'] = list_user_created_at

In [62]:
completed_dict

{'t_id': [763431264577728512,
  763431264061755392,
  763431265370505216,
  763431265743802368,
  763431266171645952,
  763431266158911488,
  763431266347782145,
  763431266150526976,
  763431267362672640,
  763431267471880192,
  763431267744505856,
  763431267668938752,
  763431267501125632,
  763431268289683457,
  763431268587626496,
  763431268402987008,
  763431267929038848,
  763431269233491968,
  763431270022012930,
  763431270063931392,
  763431270433030144,
  763431269917036544,
  763431270013558784,
  763431268709175296,
  763431271301312512,
  763431271569784832,
  763431271754297344,
  763431271817220100,
  763431271989063680,
  763431271573913600,
  763431274728030208,
  763431276019789824,
  763431275944443908,
  763431276477083648,
  763431276317663232,
  763431276153933824,
  763431276762296320,
  763431276749594624,
  763431277156368384,
  763431276783161344,
  763431276741218305,
  763431277609484288,
  763431277915627521,
  763431278062374912,
  763431278066606081,
  

In [63]:
normal_tweets = pd.DataFrame.from_dict(completed_dict)
normal_tweets

,t_id,t_text,t_created_at,t_hashtags,t_mentions,t_urls,t_source,t_lang,t_favorite_count,t_retweet_count,...,user_verified,user_lang,followers_count,user_status_count,user_friend_count,user_favorites_count,user_listed_count,user_time_zone,user_protected,user_created_at
0,763431264577728512,"RT @jasonfekete: Great read here: In 2007, Tru...",Wed Aug 10 17:46:11 +0000 2016,[],"[{'screen_name': 'jasonfekete', 'name': 'Jason...","[{'url': 'https://t.co/yM6JGjrWbD', 'expanded_...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,0,8,...,False,None,3267,24436,2662,4549,6,None,False,Sun Oct 09 03:30:37 +0000 2011
1,763431264061755392,RT @HuffPostBiz: Trump's new economic plan is ...,Wed Aug 10 17:46:11 +0000 2016,[],"[{'screen_name': 'HuffPostBiz', 'name': 'HuffP...","[{'url': 'https://t.co/vGefdXaiDP', 'expanded_...","<a href=""http://twitter.com/download/android"" ...",en,0,40,...,False,None,4370,512610,1600,288362,172,None,False,Wed Feb 11 22:24:11 +0000 2009
2,763431265370505216,"RT @nytopinion: Forget politics, Trump is a di...",Wed Aug 10 17:46:12 +0000 2016,[],"[{'screen_name': 'nytopinion', 'name': 'New Yo...","[{'url': 'https://t.co/EErtmVZr4R', 'expanded_...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,0,1063,...,False,None,1945,10988,3736,2189,12,None,False,Wed Jan 13 15:21:19 +0000 2016
3,763431265743802368,RT @newtgingrich: Hillary Clinton snubs police...,Wed Aug 10 17:46:12 +0000 2016,[],"[{'screen_name': 'newtgingrich', 'name': 'Newt...","[{'url': 'https://t.co/5b2xCcWg9l', 'expanded_...","<a href=""http://twitter.com/download/iphone"" r...",en,0,557,...,False,None,2170,101395,1880,205578,22,None,False,Wed Feb 12 21:06:43 +0000 2014
4,763431266171645952,"RT @dredeyedick: If Trump was ""joking"", why sa...",Wed Aug 10 17:46:12 +0000 2016,[],"[{'screen_name': 'dredeyedick', 'name': 'Dave ...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,2,...,False,None,560,30694,1917,27289,185,None,False,Tue Apr 26 06:24:07 +0000 2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,763431290595057665,RT @BeauWillimon: For 3 mil @NRA votes Trump d...,Wed Aug 10 17:46:18 +0000 2016,[],"[{'screen_name': 'BeauWillimon', 'name': 'Beau...",[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,0,29,...,False,None,636,4042,150,1651,9,None,False,Mon Aug 04 15:27:21 +0000 2014
96,763431290729291776,On dirait la maison d'Amityville! 😂 https://t....,Wed Aug 10 17:46:18 +0000 2016,[],[],"[{'url': 'https://t.co/yt2XLrlrTm', 'expanded_...","<a href=""http://twitter.com/download/iphone"" r...",fr,0,0,...,False,None,391,53976,1041,32597,39,None,False,Tue Sep 11 12:55:19 +0000 2012
97,763431290834153472,RT @TeaPainUSA: The feller that wants to shoot...,Wed Aug 10 17:46:18 +0000 2016,[],"[{'screen_name': 'TeaPainUSA', 'name': 'Tea Pa...",[],"<a href=""http://twitter.com/download/android"" ...",en,0,529,...,False,None,1223,51361,2011,13991,40,None,False,Sun May 08 16:18:26 +0000 2011
98,763431290880167936,"RT @KaydeeKing: HRC at debate: ""Name one time ...",Wed Aug 10 17:46:18 +0000 2016,[],"[{'screen_name': 'KaydeeKing', 'name': 'Kaydee...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,71,...,False,None,540,3039,2608,5320,23,None,False,Sun Oct 04 22:40:40 +0000 2009


In [64]:
normal_tweets.to_csv('normal_tweets.csv')

In [ ]:
#Okay, whenever we can move forward, I should just run it from after the last tweet id I was able to extract info from
#(763431291723378689)

In [ ]:
#Seems like it's already working

In [66]:
normal_id[normal_id['tweet_id'] == 763431291723378689]

,tweet_id
238,763431291723378689


In [68]:
list_t_id = []
list_t_text = []
list_t_created_at = []
list_t_hashtags = []
list_t_mentions = []
list_t_urls = []
list_t_source = []
list_t_lang = []
list_t_favorite_count = []
list_t_retweet_count = []
list_t_retweeted = []
list_t_retweet_status_id = []
list_t_in_reply_to_status_id = []
    
list_user_id = []
list_user_name = []
list_user_handle = []
list_user_location = []
list_user_desc = []
list_user_verified = []
list_user_lang = []
list_user_followers_count= []
list_user_status_count = []
list_user_friend_count = []
list_user_favorites_count = []
list_user_listed_count = []
list_user_time_zone = []
list_user_protected = []
list_user_created_at = []

for id in normal_id[239:1000]['tweet_id']:
    def create_url(id):
    #keyword, start_date, end_date, max_results = 10 #goes into the above

    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
        search_url = 'https://api.twitter.com/1.1/statuses/show.json?id='+str(id)
    #change params based on the endpoint you are using
        query_params = {#'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    #'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
        return (search_url, query_params)

    url = create_url(id)

#keyword, start_time, end_time, max_results
    def connect_to_endpoint(url, headers, params, next_token = None):
        params['next_token'] = next_token   #params object received from create_url function
        response = requests.request("GET", url, headers = headers, params = params)
        print("Endpoint Response Code: " + str(response.status_code))
        if response.status_code != 200:
           # raise Exception(response.status_code, response.text)
            pass
        return response.json()

    json_response = connect_to_endpoint(url[0], headers, url[1])
    if len(json_response) > 1:
        t_user= json_response['user']
        #don't need to push the above, just for below
        t_id = json_response['id']
        list_t_id.append(t_id)
    
        t_text = json_response['text']
        list_t_text.append(t_text)
    
        t_created_at = json_response['created_at']
        list_t_created_at.append(t_created_at)
    
        t_hashtags = json_response['entities']['hashtags']
        list_t_hashtags.append(t_hashtags)
    
        t_mentions = json_response['entities']['user_mentions']
        list_t_mentions.append(t_mentions)
    
        t_urls = json_response['entities']['urls']
        list_t_urls.append(t_urls)
    
        t_source = json_response['source']
        list_t_source.append(t_source)
    
        t_lang = json_response['lang']
        list_t_lang.append(t_lang)
    
        t_favorite_count = json_response['favorite_count']
        list_t_favorite_count.append(t_favorite_count)
    
        t_retweet_count = json_response['retweet_count']
        list_t_retweet_count.append(t_retweet_count)
    
        t_retweeted = json_response['retweeted']
        list_t_retweeted.append(t_retweeted)
        
        if 'retweeted_status' in json_response:
            t_retweet_status_id = json_response['retweeted_status']['id']
            list_t_retweet_status_id.append(t_retweet_status_id)
        else:
            list_t_retweet_status_id.append("None")
    
        t_in_reply_to_status_id = json_response['in_reply_to_status_id']
        list_t_in_reply_to_status_id.append(t_in_reply_to_status_id)
    
        user_id = t_user['id']
        list_user_id.append(user_id)
    
        user_name = t_user['name']
        list_user_name.append(user_name)
    
        user_handle = t_user['screen_name']
        list_user_handle.append(user_handle)
    
        user_location = t_user['location']
        list_user_location.append(user_location)
    
        user_desc = t_user['description']
        list_user_desc.append(user_desc)
    
        user_verified = t_user['verified'] 
        list_user_verified.append(user_verified)
    
        user_lang = t_user['lang']
        list_user_lang.append(user_lang)
    
        user_followers_count= t_user['followers_count']
        list_user_followers_count.append(user_followers_count)
    
        user_status_count = t_user['statuses_count']
        list_user_status_count.append(user_status_count)
    
        user_friend_count = t_user['friends_count']
        list_user_friend_count.append(user_friend_count)
    
        user_favorites_count = t_user['favourites_count']
        list_user_favorites_count.append(user_favorites_count)
    
        user_listed_count = t_user['listed_count']
        list_user_listed_count.append(user_listed_count)
    
        user_time_zone = t_user['time_zone']
        list_user_time_zone.append(user_time_zone)
    
        user_protected = t_user['protected']
        list_user_protected.append(user_protected)
    
        user_created_at = t_user['created_at']
        list_user_created_at.append(user_created_at)
    else:
        pass

Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Co

Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Co

Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Co

In [69]:
completed_dict = {}
completed_dict['t_id'] = list_t_id
completed_dict['t_text'] = list_t_text
completed_dict['t_created_at'] = list_t_created_at
completed_dict['t_hashtags'] = list_t_hashtags
completed_dict['t_mentions'] = list_t_mentions
completed_dict['t_urls'] = list_t_urls
completed_dict['t_source'] = list_t_source
completed_dict['t_lang'] = list_t_lang
completed_dict['t_favorite_count'] = list_t_favorite_count
completed_dict['t_retweet_count'] = list_t_retweet_count
completed_dict['t_retweeted'] = list_t_retweeted
completed_dict['t_retweet_status_id'] = list_t_retweet_status_id
completed_dict['t_in_reply_to_status_id'] = list_t_in_reply_to_status_id
    
completed_dict['user_id'] = list_user_id
completed_dict['user_name'] = list_user_name
completed_dict['user_handle'] = list_user_handle
completed_dict['user_locatoin'] = list_user_location
completed_dict['user_desc'] = list_user_desc
completed_dict['user_verified'] = list_user_verified
completed_dict['user_lang'] = list_user_lang
completed_dict['followers_count'] = list_user_followers_count
completed_dict['user_status_count'] = list_user_status_count
completed_dict['user_friend_count'] = list_user_friend_count
completed_dict['user_favorites_count'] = list_user_favorites_count
completed_dict['user_listed_count'] = list_user_listed_count
completed_dict['user_time_zone'] = list_user_time_zone
completed_dict['user_protected'] = list_user_protected
completed_dict['user_created_at'] = list_user_created_at
completed_dict

{'t_id': [763431292327251973,
  763431291916210180,
  763431292423856129,
  763431292302221316,
  763431292885237760,
  763431292654477312,
  763431292939755520,
  763431293069692930,
  763431293480730624,
  763431292180520960,
  763431292880814080,
  763431293803651073,
  763431293589811200,
  763431294021701632,
  763431294734786561,
  763431295032655872,
  763431295263338496,
  763431295573721089,
  763431295619833856,
  763431294411735041,
  763431295749857284,
  763431295380623361,
  763431295930077184,
  763431295774896128,
  763431295636516864,
  763431295623897088,
  763431296207052801,
  763431296227946496,
  763431296429195264,
  763431296253227009,
  763431296643244034,
  763431297058570240,
  763431296941121536,
  763431297444352000,
  763431297582673920,
  763431297628844033,
  763431296672600065,
  763431298572562432,
  763431298606043138,
  763431299684110336,
  763431299440779264,
  763431300212543488,
  763431301114392576,
  763431301026381825,
  763431301806428160,
  

In [70]:
normal_tweets1 = pd.DataFrame.from_dict(completed_dict)
normal_tweets1

,t_id,t_text,t_created_at,t_hashtags,t_mentions,t_urls,t_source,t_lang,t_favorite_count,t_retweet_count,...,user_verified,user_lang,followers_count,user_status_count,user_friend_count,user_favorites_count,user_listed_count,user_time_zone,user_protected,user_created_at
0,763431292327251973,RT @Sandstorm1776: @redflagnews Wonder where t...,Wed Aug 10 17:46:18 +0000 2016,[],"[{'screen_name': 'Sandstorm1776', 'name': 'San...",[],"<a href=""http://twitter.com/download/android"" ...",en,0,1,...,False,None,8695,71988,9539,58019,0,None,False,Fri Aug 30 23:26:57 +0000 2013
1,763431291916210180,#DamnDaniel Back at it again with the smackdow...,Wed Aug 10 17:46:18 +0000 2016,"[{'text': 'DamnDaniel', 'indices': [0, 11]}]",[],"[{'url': 'https://t.co/3RKk8uowOR', 'expanded_...","<a href=""http://twitter.com/download/android"" ...",en,0,10,...,False,None,883,36389,46,2869,53,None,False,Tue Jun 23 18:44:38 +0000 2015
2,763431292423856129,RT @TeaPainUSA: The feller that wants to shoot...,Wed Aug 10 17:46:18 +0000 2016,[],"[{'screen_name': 'TeaPainUSA', 'name': 'Tea Pa...",[],"<a href=""http://twitter.com/#!/download/ipad"" ...",en,0,529,...,False,None,3243,157161,2921,165148,79,None,False,Sat Apr 03 22:39:44 +0000 2010
3,763431292302221316,"RT @HillaryClinton: ""The time has come for Rep...",Wed Aug 10 17:46:18 +0000 2016,[],"[{'screen_name': 'HillaryClinton', 'name': 'Hi...","[{'url': 'https://t.co/yMKCpKMC4l', 'expanded_...","<a href=""https://www.flipboard.com"" rel=""nofol...",en,0,1922,...,False,None,50,18913,303,73066,1,None,False,Fri Mar 04 22:38:03 +0000 2016
4,763431292885237760,"Still waiting, Twitter. I'm not going to surv...",Wed Aug 10 17:46:18 +0000 2016,[],[],"[{'url': 'https://t.co/QMUYvtONR3', 'expanded_...","<a href=""http://twitter.com/download/android"" ...",en,0,0,...,False,None,66,3534,229,13769,6,None,False,Fri Oct 02 01:58:50 +0000 2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,763431372522266624,RT @kurteichenwald: A guy at Trump rally yelle...,Wed Aug 10 17:46:37 +0000 2016,[],"[{'screen_name': 'kurteichenwald', 'name': 'Ku...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,1290,...,False,None,720,82384,806,43730,70,None,False,Tue Oct 01 04:42:31 +0000 2013
351,763431372761526272,RT @Bernard_Monot: Soutiens #Clinton: père pro...,Wed Aug 10 17:46:37 +0000 2016,"[{'text': 'Clinton', 'indices': [28, 36]}, {'t...","[{'screen_name': 'Bernard_Monot', 'name': 'Ber...",[],"<a href=""http://twitter.com/download/android"" ...",fr,0,24,...,False,None,4911,105819,4269,74068,72,None,False,Fri Nov 14 18:33:12 +0000 2014
352,763431372140666880,"DEMOCRATS GOOD AT PROPAGANDA. ALL U "" JUMPING...",Wed Aug 10 17:46:37 +0000 2016,[],[],"[{'url': 'https://t.co/ynf1DYejmc', 'expanded_...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,0,0,...,False,None,29,3922,272,2188,0,None,False,Fri Mar 07 03:50:23 +0000 2014
353,763431373172576257,RT @JonathanMerritt: Trump-loving Christians s...,Wed Aug 10 17:46:37 +0000 2016,[],"[{'screen_name': 'JonathanMerritt', 'name': 'J...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,176,...,True,None,22484,128573,1025,105297,381,None,False,Wed Sep 30 04:39:18 +0000 2009


In [71]:
normal_tweets1.to_csv('normal_tweets1.csv')

In [72]:
normal_id[normal_id['tweet_id'] == 763431373445091328] #Makes sense if it ended on a few non-accounts
#The number of accounts that either got suspended or don't work is pretty astounding - will bias results?
#Or arguably shows they're not nefarious because they're not suspended?

,tweet_id
997,763431373445091328


In [103]:
list_t_id = []
list_t_text = []
list_t_created_at = []
list_t_hashtags = []
list_t_mentions = []
list_t_urls = []
list_t_source = []
list_t_lang = []
list_t_favorite_count = []
list_t_retweet_count = []
list_t_retweeted = []
list_t_retweet_status_id = []
list_t_in_reply_to_status_id = []
    
list_user_id = []
list_user_name = []
list_user_handle = []
list_user_location = []
list_user_desc = []
list_user_verified = []
list_user_lang = []
list_user_followers_count= []
list_user_status_count = []
list_user_friend_count = []
list_user_favorites_count = []
list_user_listed_count = []
list_user_time_zone = []
list_user_protected = []
list_user_created_at = []

for id in normal_id[4895:5000]['tweet_id']:
    def create_url(id):
    #keyword, start_date, end_date, max_results = 10 #goes into the above

    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
        search_url = 'https://api.twitter.com/1.1/statuses/show.json?id='+str(id)
    #change params based on the endpoint you are using
        query_params = {#'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    #'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
        return (search_url, query_params)

    url = create_url(id)

#keyword, start_time, end_time, max_results
    def connect_to_endpoint(url, headers, params, next_token = None):
        params['next_token'] = next_token   #params object received from create_url function
        response = requests.request("GET", url, headers = headers, params = params)
        print("Endpoint Response Code: " + str(response.status_code))
        if response.status_code != 200:
           # raise Exception(response.status_code, response.text)
            pass
        return response.json()

    json_response = connect_to_endpoint(url[0], headers, url[1])
    if len(json_response) > 1:
        t_user= json_response['user']
        #don't need to push the above, just for below
        t_id = json_response['id']
        list_t_id.append(t_id)
    
        t_text = json_response['text']
        list_t_text.append(t_text)
    
        t_created_at = json_response['created_at']
        list_t_created_at.append(t_created_at)
    
        t_hashtags = json_response['entities']['hashtags']
        list_t_hashtags.append(t_hashtags)
    
        t_mentions = json_response['entities']['user_mentions']
        list_t_mentions.append(t_mentions)
    
        t_urls = json_response['entities']['urls']
        list_t_urls.append(t_urls)
    
        t_source = json_response['source']
        list_t_source.append(t_source)
    
        t_lang = json_response['lang']
        list_t_lang.append(t_lang)
    
        t_favorite_count = json_response['favorite_count']
        list_t_favorite_count.append(t_favorite_count)
    
        t_retweet_count = json_response['retweet_count']
        list_t_retweet_count.append(t_retweet_count)
    
        t_retweeted = json_response['retweeted']
        list_t_retweeted.append(t_retweeted)
        
        if 'retweeted_status' in json_response:
            t_retweet_status_id = json_response['retweeted_status']['id']
            list_t_retweet_status_id.append(t_retweet_status_id)
        else:
            list_t_retweet_status_id.append("None")
    
        t_in_reply_to_status_id = json_response['in_reply_to_status_id']
        list_t_in_reply_to_status_id.append(t_in_reply_to_status_id)
    
        user_id = t_user['id']
        list_user_id.append(user_id)
    
        user_name = t_user['name']
        list_user_name.append(user_name)
    
        user_handle = t_user['screen_name']
        list_user_handle.append(user_handle)
    
        user_location = t_user['location']
        list_user_location.append(user_location)
    
        user_desc = t_user['description']
        list_user_desc.append(user_desc)
    
        user_verified = t_user['verified'] 
        list_user_verified.append(user_verified)
    
        user_lang = t_user['lang']
        list_user_lang.append(user_lang)
    
        user_followers_count= t_user['followers_count']
        list_user_followers_count.append(user_followers_count)
    
        user_status_count = t_user['statuses_count']
        list_user_status_count.append(user_status_count)
    
        user_friend_count = t_user['friends_count']
        list_user_friend_count.append(user_friend_count)
    
        user_favorites_count = t_user['favourites_count']
        list_user_favorites_count.append(user_favorites_count)
    
        user_listed_count = t_user['listed_count']
        list_user_listed_count.append(user_listed_count)
    
        user_time_zone = t_user['time_zone']
        list_user_time_zone.append(user_time_zone)
    
        user_protected = t_user['protected']
        list_user_protected.append(user_protected)
    
        user_created_at = t_user['created_at']
        list_user_created_at.append(user_created_at)
    else:
        pass

Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Co

Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Co

Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Co

Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Code: 429
Endpoint Response Co

In [104]:
completed_dict = {}
completed_dict['t_id'] = list_t_id
completed_dict['t_text'] = list_t_text
completed_dict['t_created_at'] = list_t_created_at
completed_dict['t_hashtags'] = list_t_hashtags
completed_dict['t_mentions'] = list_t_mentions
completed_dict['t_urls'] = list_t_urls
completed_dict['t_source'] = list_t_source
completed_dict['t_lang'] = list_t_lang
completed_dict['t_favorite_count'] = list_t_favorite_count
completed_dict['t_retweet_count'] = list_t_retweet_count
completed_dict['t_retweeted'] = list_t_retweeted
completed_dict['t_retweet_status_id'] = list_t_retweet_status_id
completed_dict['t_in_reply_to_status_id'] = list_t_in_reply_to_status_id
    
completed_dict['user_id'] = list_user_id
completed_dict['user_name'] = list_user_name
completed_dict['user_handle'] = list_user_handle
completed_dict['user_locatoin'] = list_user_location
completed_dict['user_desc'] = list_user_desc
completed_dict['user_verified'] = list_user_verified
completed_dict['user_lang'] = list_user_lang
completed_dict['followers_count'] = list_user_followers_count
completed_dict['user_status_count'] = list_user_status_count
completed_dict['user_friend_count'] = list_user_friend_count
completed_dict['user_favorites_count'] = list_user_favorites_count
completed_dict['user_listed_count'] = list_user_listed_count
completed_dict['user_time_zone'] = list_user_time_zone
completed_dict['user_protected'] = list_user_protected
completed_dict['user_created_at'] = list_user_created_at

normal_tweets8 = pd.DataFrame.from_dict(completed_dict)
normal_tweets8

,t_id,t_text,t_created_at,t_hashtags,t_mentions,t_urls,t_source,t_lang,t_favorite_count,t_retweet_count,...,user_verified,user_lang,followers_count,user_status_count,user_friend_count,user_favorites_count,user_listed_count,user_time_zone,user_protected,user_created_at
0,763431703759192064,RT @brunobonsanti: Trump sugeriu mesmo que a g...,Wed Aug 10 17:47:56 +0000 2016,[],"[{'screen_name': 'brunobonsanti', 'name': 'Bon...","[{'url': 'https://t.co/o5qpKAXj13', 'expanded_...","<a href=""http://twitter.com/download/android"" ...",pt,0,29,...,False,None,307,5792,2076,6560,0,None,False,Fri May 06 15:54:50 +0000 2011
1,763431703759249409,RT @gugachacra: Trump ultrapassou qualquer lim...,Wed Aug 10 17:47:56 +0000 2016,[],"[{'screen_name': 'gugachacra', 'name': 'Guga C...",[],"<a href=""http://twitter.com/download/android"" ...",pt,0,106,...,False,None,307,5792,2076,6560,0,None,False,Fri May 06 15:54:50 +0000 2011
2,763431704417734657,@speakerryan has #betterway . #trump Joke abou...,Wed Aug 10 17:47:56 +0000 2016,"[{'text': 'betterway', 'indices': [17, 27]}, {...","[{'screen_name': 'SpeakerRyan', 'name': 'Paul ...",[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,0,0,...,False,None,157,5040,390,109,24,None,False,Wed Jun 29 16:04:47 +0000 2016
3,763431704359006209,.@tazgezwitscher @WAinetter Im schlimmsten Fal...,Wed Aug 10 17:47:56 +0000 2016,[],"[{'screen_name': 'tazgezwitscher', 'name': 'ta...",[],"<a href=""http://twitter.com/download/android"" ...",de,1,0,...,False,None,3404,34617,602,11896,109,None,False,Mon Jul 20 10:18:25 +0000 2009
4,763431704157487105,@psgcindy @nypost https://t.co/yxc9BXierN,Wed Aug 10 17:47:56 +0000 2016,[],"[{'screen_name': 'nypost', 'name': 'New York P...","[{'url': 'https://t.co/yxc9BXierN', 'expanded_...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",und,0,0,...,False,None,269,29970,2018,19257,8,None,False,Sat Feb 26 22:18:19 +0000 2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,763431802631491584,Judicial Watch Uncovers New Batch of Hillary C...,Wed Aug 10 17:48:20 +0000 2016,[],"[{'screen_name': 'JohnKStahlUSA', 'name': 'Joh...","[{'url': 'https://t.co/fWjDFJAjWW', 'expanded_...","<a href=""http://twitter.com/download/android"" ...",en,0,0,...,False,None,681,88516,1358,131420,16,None,False,Mon Jan 12 03:27:07 +0000 2015
395,763431802870595585,@realDonaldTrump and we r counting on u 2 defe...,Wed Aug 10 17:48:20 +0000 2016,[],"[{'screen_name': 'realDonaldTrump', 'name': 'D...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,0,...,False,None,109,1491,211,260,1,None,False,Sun Jul 24 17:58:37 +0000 2016
396,763431803055112192,RT @Slate: Ivanka Trump’s clothing line is des...,Wed Aug 10 17:48:20 +0000 2016,[],"[{'screen_name': 'Slate', 'name': 'Slate', 'id...","[{'url': 'https://t.co/mh8TF4WJ36', 'expanded_...","<a href=""http://twitter.com/download/iphone"" r...",en,0,72,...,False,None,118,3738,1035,138,0,None,False,Sat Oct 31 00:14:28 +0000 2015
397,763431803474640896,RT @FoxNews: HILLARY FAVOR-TRADING? New emails...,Wed Aug 10 17:48:20 +0000 2016,[],"[{'screen_name': 'FoxNews', 'name': 'Fox News'...","[{'url': 'https://t.co/9J4x7vqePD', 'expanded_...","<a href=""http://twitter.com/download/iphone"" r...",en,0,607,...,False,None,954,58179,749,25618,23,None,False,Fri May 20 21:18:39 +0000 2011


In [106]:
normal_tweets8.to_csv('normal_tweets8.csv')

In [105]:
normal_id[normal_id['tweet_id'] == 763431803331960832] #Last one from this filter

,tweet_id
4894,763431803331960832


In [101]:
normal_id.iloc[3097] 
#Double-checked with a longer enpoint response error message and 403 means the account was suspended -
#i.e. specifically this account here was suspended

tweet_id    763431603989282817
Name: 3097, dtype: int64

In [108]:
filter2 = np.loadtxt(fname='election-filter2.txt', dtype=int)
#filter3 #This is a LOT of id #s
normal_id_2 = pd.DataFrame(filter2, columns = ['tweet_id'])
normal_id_2
#normal_id_2.to_csv('normal_ids2.csv')
#for id in normal_id_3[:2000]['tweet_id']:
#    print(id)

,tweet_id
0,782631555172827137
1,782631556573638656
2,782631555495755776
3,782631557051904000
4,782631557261512705
...,...
49999995,787694963907760128
49999996,787694963853107200
49999997,787694963844841473
49999998,787694963484135425


In [102]:
#Double-checked with a longer enpoint response error message and 403 means the account was suspended

Endpoint Response Code: 403


Exception: (403, '{"errors":[{"code":63,"message":"User has been suspended."}]}')

In [141]:
list_t_id = []
list_t_text = []
list_t_created_at = []
list_t_hashtags = []
list_t_mentions = []
list_t_urls = []
list_t_source = []
list_t_lang = []
list_t_favorite_count = []
list_t_retweet_count = []
list_t_retweeted = []
list_t_retweet_status_id = []
list_t_in_reply_to_status_id = []
    
list_user_id = []
list_user_name = []
list_user_handle = []
list_user_location = []
list_user_desc = []
list_user_verified = []
list_user_lang = []
list_user_followers_count= []
list_user_status_count = []
list_user_friend_count = []
list_user_favorites_count = []
list_user_listed_count = []
list_user_time_zone = []
list_user_protected = []
list_user_created_at = []

for id in normal_id_2[4499:5000]['tweet_id']:
    def create_url(id):
    #keyword, start_date, end_date, max_results = 10 #goes into the above

    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
        search_url = 'https://api.twitter.com/1.1/statuses/show.json?id='+str(id)
    #change params based on the endpoint you are using
        query_params = {#'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    #'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
        return (search_url, query_params)

    url = create_url(id)

#keyword, start_time, end_time, max_results
    def connect_to_endpoint(url, headers, params, next_token = None):
        params['next_token'] = next_token   #params object received from create_url function
        response = requests.request("GET", url, headers = headers, params = params)
        print("Endpoint Response Code: " + str(response.status_code))
        if response.status_code != 200:
           # raise Exception(response.status_code, response.text)
            pass
        return response.json()

    json_response = connect_to_endpoint(url[0], headers, url[1])
    if len(json_response) > 1:
        t_user= json_response['user']
        #don't need to push the above, just for below
        t_id = json_response['id']
        list_t_id.append(t_id)
    
        t_text = json_response['text']
        list_t_text.append(t_text)
    
        t_created_at = json_response['created_at']
        list_t_created_at.append(t_created_at)
    
        t_hashtags = json_response['entities']['hashtags']
        list_t_hashtags.append(t_hashtags)
    
        t_mentions = json_response['entities']['user_mentions']
        list_t_mentions.append(t_mentions)
    
        t_urls = json_response['entities']['urls']
        list_t_urls.append(t_urls)
    
        t_source = json_response['source']
        list_t_source.append(t_source)
    
        t_lang = json_response['lang']
        list_t_lang.append(t_lang)
    
        t_favorite_count = json_response['favorite_count']
        list_t_favorite_count.append(t_favorite_count)
    
        t_retweet_count = json_response['retweet_count']
        list_t_retweet_count.append(t_retweet_count)
    
        t_retweeted = json_response['retweeted']
        list_t_retweeted.append(t_retweeted)
        
        if 'retweeted_status' in json_response:
            t_retweet_status_id = json_response['retweeted_status']['id']
            list_t_retweet_status_id.append(t_retweet_status_id)
        else:
            list_t_retweet_status_id.append("None")
    
        t_in_reply_to_status_id = json_response['in_reply_to_status_id']
        list_t_in_reply_to_status_id.append(t_in_reply_to_status_id)
    
        user_id = t_user['id']
        list_user_id.append(user_id)
    
        user_name = t_user['name']
        list_user_name.append(user_name)
    
        user_handle = t_user['screen_name']
        list_user_handle.append(user_handle)
    
        user_location = t_user['location']
        list_user_location.append(user_location)
    
        user_desc = t_user['description']
        list_user_desc.append(user_desc)
    
        user_verified = t_user['verified'] 
        list_user_verified.append(user_verified)
    
        user_lang = t_user['lang']
        list_user_lang.append(user_lang)
    
        user_followers_count= t_user['followers_count']
        list_user_followers_count.append(user_followers_count)
    
        user_status_count = t_user['statuses_count']
        list_user_status_count.append(user_status_count)
    
        user_friend_count = t_user['friends_count']
        list_user_friend_count.append(user_friend_count)
    
        user_favorites_count = t_user['favourites_count']
        list_user_favorites_count.append(user_favorites_count)
    
        user_listed_count = t_user['listed_count']
        list_user_listed_count.append(user_listed_count)
    
        user_time_zone = t_user['time_zone']
        list_user_time_zone.append(user_time_zone)
    
        user_protected = t_user['protected']
        list_user_protected.append(user_protected)
    
        user_created_at = t_user['created_at']
        list_user_created_at.append(user_created_at)
    else:
        pass

Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Co

Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Co

In [142]:
completed_dict = {}
completed_dict['t_id'] = list_t_id
completed_dict['t_text'] = list_t_text
completed_dict['t_created_at'] = list_t_created_at
completed_dict['t_hashtags'] = list_t_hashtags
completed_dict['t_mentions'] = list_t_mentions
completed_dict['t_urls'] = list_t_urls
completed_dict['t_source'] = list_t_source
completed_dict['t_lang'] = list_t_lang
completed_dict['t_favorite_count'] = list_t_favorite_count
completed_dict['t_retweet_count'] = list_t_retweet_count
completed_dict['t_retweeted'] = list_t_retweeted
completed_dict['t_retweet_status_id'] = list_t_retweet_status_id
completed_dict['t_in_reply_to_status_id'] = list_t_in_reply_to_status_id
    
completed_dict['user_id'] = list_user_id
completed_dict['user_name'] = list_user_name
completed_dict['user_handle'] = list_user_handle
completed_dict['user_locatoin'] = list_user_location
completed_dict['user_desc'] = list_user_desc
completed_dict['user_verified'] = list_user_verified
completed_dict['user_lang'] = list_user_lang
completed_dict['followers_count'] = list_user_followers_count
completed_dict['user_status_count'] = list_user_status_count
completed_dict['user_friend_count'] = list_user_friend_count
completed_dict['user_favorites_count'] = list_user_favorites_count
completed_dict['user_listed_count'] = list_user_listed_count
completed_dict['user_time_zone'] = list_user_time_zone
completed_dict['user_protected'] = list_user_protected
completed_dict['user_created_at'] = list_user_created_at

normal_tweets_f2_8 = pd.DataFrame.from_dict(completed_dict)
normal_tweets_f2_8

,t_id,t_text,t_created_at,t_hashtags,t_mentions,t_urls,t_source,t_lang,t_favorite_count,t_retweet_count,...,user_verified,user_lang,followers_count,user_status_count,user_friend_count,user_favorites_count,user_listed_count,user_time_zone,user_protected,user_created_at
0,782631990956847104,RT @Left_Blog: Definitely RT if you agree: Hil...,Sun Oct 02 17:23:01 +0000 2016,[],"[{'screen_name': 'Left_Blog', 'name': 'Left', ...","[{'url': 'https://t.co/00lg6C64Cf', 'expanded_...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,0,193,...,False,None,288,6417,1245,29092,8,None,False,Wed Feb 16 22:25:01 +0000 2011
1,782631990780493824,"Donald Trump's history with women: adultery, o...",Sun Oct 02 17:23:01 +0000 2016,[],[],"[{'url': 'https://t.co/iNsMfb1U4U', 'expanded_...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,1,0,...,False,None,2105,149731,3852,85051,32,None,False,Tue Sep 27 13:11:13 +0000 2011
2,782631990503673856,RT @BettyBowers: TRUMP: “I don’t pay taxes bec...,Sun Oct 02 17:23:01 +0000 2016,[],"[{'screen_name': 'BettyBowers', 'name': 'Mrs. ...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,337,...,False,None,2756,92638,2813,529,46,None,False,Wed Apr 02 00:26:44 +0000 2008
3,782631991074185216,RT @TaraSetmayer: Trump's new DC hotel vandali...,Sun Oct 02 17:23:01 +0000 2016,[],"[{'screen_name': 'TaraSetmayer', 'name': 'Tara...",[],"<a href=""http://twitter.com/#!/download/ipad"" ...",en,0,2016,...,False,None,45,26383,35,798,26,None,False,Wed Mar 28 22:21:27 +0000 2012
4,782631991116050432,RT @BettyBowers: Donald Trump doesn't pay any ...,Sun Oct 02 17:23:01 +0000 2016,[],"[{'screen_name': 'BettyBowers', 'name': 'Mrs. ...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,380,...,False,None,198,7540,284,60308,8,None,False,Tue Mar 10 02:59:10 +0000 2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,782632038008360960,Nelly is a genius by Donald Trump's standards,Sun Oct 02 17:23:13 +0000 2016,[],[],[],"<a href=""http://twitter.com/download/android"" ...",en,0,0,...,False,None,1932,55426,3588,483332,23,None,False,Wed Nov 07 05:47:49 +0000 2012
231,782632038117638144,RT @DrTomMartinPhD: Donald Trump Yo-Yo Tricks ...,Sun Oct 02 17:23:13 +0000 2016,"[{'text': 'msnbc', 'indices': [63, 69]}]","[{'screen_name': 'DrTomMartinPhD', 'name': 'Dr...",[],"<a href=""https://mobile.twitter.com"" rel=""nofo...",en,0,2,...,False,None,1869,114904,1720,67532,68,None,False,Tue Feb 09 20:45:17 +0000 2016
232,782632037823844352,RT @MarkRuffalo: Where I come from this makes ...,Sun Oct 02 17:23:13 +0000 2016,[],"[{'screen_name': 'MarkRuffalo', 'name': 'Mark ...","[{'url': 'https://t.co/9f4ND0DrVI', 'expanded_...","<a href=""http://twitter.com/download/iphone"" r...",en,0,379,...,False,None,41,10081,406,18718,4,None,False,Tue Dec 10 20:03:26 +0000 2013
233,782632033449152514,So *WHY* are so many people willing to let #Tr...,Sun Oct 02 17:23:12 +0000 2016,"[{'text': 'Trump', 'indices': [43, 49]}, {'tex...",[],"[{'url': 'https://t.co/v87dwMDGxf', 'expanded_...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,2,0,...,False,None,1012,291494,720,266481,150,None,False,Fri Apr 24 00:44:01 +0000 2015


In [144]:
normal_tweets_f2_8.to_csv('normal_tweets_f2_8.csv')

In [143]:
normal_id_2[normal_id_2['tweet_id'] == 782632038050299904] 

,tweet_id
4997,782632038050299904


In [316]:
filter3 = np.loadtxt(fname='election-filter3.txt', dtype=int)
#filter3 #This is a LOT of id #s
normal_id_3 = pd.DataFrame(filter3, columns = ['tweet_id'])
normal_id_3
#normal_id_3.to_csv('normal_ids3.csv')
#for id in normal_id_3[:2000]['tweet_id']:
#    print(id)

,tweet_id
0,787694963458846720
1,787694963786063872
2,787694963886788609
3,787694964390129665
4,787694963677028353
...,...
49999995,777559127094505472
49999996,777559127056723969
49999997,777559127174225920
49999998,777559127442665472


In [318]:
list_t_id = []
list_t_text = []
list_t_created_at = []
list_t_hashtags = []
list_t_mentions = []
list_t_urls = []
list_t_source = []
list_t_lang = []
list_t_favorite_count = []
list_t_retweet_count = []
list_t_retweeted = []
list_t_retweet_status_id = []
list_t_in_reply_to_status_id = []
    
list_user_id = []
list_user_name = []
list_user_handle = []
list_user_location = []
list_user_desc = []
list_user_verified = []
list_user_lang = []
list_user_followers_count= []
list_user_status_count = []
list_user_friend_count = []
list_user_favorites_count = []
list_user_listed_count = []
list_user_time_zone = []
list_user_protected = []
list_user_created_at = []

for id in normal_id_3[4499:5000]['tweet_id']:
    def create_url(id):
    #keyword, start_date, end_date, max_results = 10 #goes into the above

    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
        search_url = 'https://api.twitter.com/1.1/statuses/show.json?id='+str(id)
    #change params based on the endpoint you are using
        query_params = {#'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    #'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
        return (search_url, query_params)

    url = create_url(id)

#keyword, start_time, end_time, max_results
    def connect_to_endpoint(url, headers, params, next_token = None):
        params['next_token'] = next_token   #params object received from create_url function
        response = requests.request("GET", url, headers = headers, params = params)
        print("Endpoint Response Code: " + str(response.status_code))
        if response.status_code != 200:
           # raise Exception(response.status_code, response.text)
            pass
        return response.json()

    json_response = connect_to_endpoint(url[0], headers, url[1])
    if len(json_response) > 1:
        t_user= json_response['user']
        #don't need to push the above, just for below
        t_id = json_response['id']
        list_t_id.append(t_id)
    
        t_text = json_response['text']
        list_t_text.append(t_text)
    
        t_created_at = json_response['created_at']
        list_t_created_at.append(t_created_at)
    
        t_hashtags = json_response['entities']['hashtags']
        list_t_hashtags.append(t_hashtags)
    
        t_mentions = json_response['entities']['user_mentions']
        list_t_mentions.append(t_mentions)
    
        t_urls = json_response['entities']['urls']
        list_t_urls.append(t_urls)
    
        t_source = json_response['source']
        list_t_source.append(t_source)
    
        t_lang = json_response['lang']
        list_t_lang.append(t_lang)
    
        t_favorite_count = json_response['favorite_count']
        list_t_favorite_count.append(t_favorite_count)
    
        t_retweet_count = json_response['retweet_count']
        list_t_retweet_count.append(t_retweet_count)
    
        t_retweeted = json_response['retweeted']
        list_t_retweeted.append(t_retweeted)
        
        if 'retweeted_status' in json_response:
            t_retweet_status_id = json_response['retweeted_status']['id']
            list_t_retweet_status_id.append(t_retweet_status_id)
        else:
            list_t_retweet_status_id.append("None")
    
        t_in_reply_to_status_id = json_response['in_reply_to_status_id']
        list_t_in_reply_to_status_id.append(t_in_reply_to_status_id)
    
        user_id = t_user['id']
        list_user_id.append(user_id)
    
        user_name = t_user['name']
        list_user_name.append(user_name)
    
        user_handle = t_user['screen_name']
        list_user_handle.append(user_handle)
    
        user_location = t_user['location']
        list_user_location.append(user_location)
    
        user_desc = t_user['description']
        list_user_desc.append(user_desc)
    
        user_verified = t_user['verified'] 
        list_user_verified.append(user_verified)
    
        user_lang = t_user['lang']
        list_user_lang.append(user_lang)
    
        user_followers_count= t_user['followers_count']
        list_user_followers_count.append(user_followers_count)
    
        user_status_count = t_user['statuses_count']
        list_user_status_count.append(user_status_count)
    
        user_friend_count = t_user['friends_count']
        list_user_friend_count.append(user_friend_count)
    
        user_favorites_count = t_user['favourites_count']
        list_user_favorites_count.append(user_favorites_count)
    
        user_listed_count = t_user['listed_count']
        list_user_listed_count.append(user_listed_count)
    
        user_time_zone = t_user['time_zone']
        list_user_time_zone.append(user_time_zone)
    
        user_protected = t_user['protected']
        list_user_protected.append(user_protected)
    
        user_created_at = t_user['created_at']
        list_user_created_at.append(user_created_at)
    else:
        pass

Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Co

Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Co

In [319]:
completed_dict = {}
completed_dict['t_id'] = list_t_id
completed_dict['t_text'] = list_t_text
completed_dict['t_created_at'] = list_t_created_at
completed_dict['t_hashtags'] = list_t_hashtags
completed_dict['t_mentions'] = list_t_mentions
completed_dict['t_urls'] = list_t_urls
completed_dict['t_source'] = list_t_source
completed_dict['t_lang'] = list_t_lang
completed_dict['t_favorite_count'] = list_t_favorite_count
completed_dict['t_retweet_count'] = list_t_retweet_count
completed_dict['t_retweeted'] = list_t_retweeted
completed_dict['t_retweet_status_id'] = list_t_retweet_status_id
completed_dict['t_in_reply_to_status_id'] = list_t_in_reply_to_status_id
    
completed_dict['user_id'] = list_user_id
completed_dict['user_name'] = list_user_name
completed_dict['user_handle'] = list_user_handle
completed_dict['user_locatoin'] = list_user_location
completed_dict['user_desc'] = list_user_desc
completed_dict['user_verified'] = list_user_verified
completed_dict['user_lang'] = list_user_lang
completed_dict['followers_count'] = list_user_followers_count
completed_dict['user_status_count'] = list_user_status_count
completed_dict['user_friend_count'] = list_user_friend_count
completed_dict['user_favorites_count'] = list_user_favorites_count
completed_dict['user_listed_count'] = list_user_listed_count
completed_dict['user_time_zone'] = list_user_time_zone
completed_dict['user_protected'] = list_user_protected
completed_dict['user_created_at'] = list_user_created_at

normal_tweets_f3_9 = pd.DataFrame.from_dict(completed_dict)
normal_tweets_f3_9

,t_id,t_text,t_created_at,t_hashtags,t_mentions,t_urls,t_source,t_lang,t_favorite_count,t_retweet_count,...,user_verified,user_lang,followers_count,user_status_count,user_friend_count,user_favorites_count,user_listed_count,user_time_zone,user_protected,user_created_at
0,787695338526117888,RT @CitizenCohn: This morning's attempt to wal...,Sun Oct 16 16:42:58 +0000 2016,[],"[{'screen_name': 'CitizenCohn', 'name': 'Jonat...","[{'url': 'https://t.co/5LwFW7KIDQ', 'expanded_...","<a href=""http://twitter.com/download/iphone"" r...",en,0,14,...,False,None,495,78726,691,137707,18,None,False,Sat Nov 28 23:06:46 +0000 2015
1,787695338140434432,"@cockdiver69 Actually, Mr. Trump, I think you ...",Sun Oct 16 16:42:57 +0000 2016,[],[],[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,0,0,...,False,None,26159,121120,614,27118,77,None,False,Sat Mar 28 15:35:45 +0000 2009
2,787695338739990528,RT @missie_88: She had a pant suit on!Did u se...,Sun Oct 16 16:42:58 +0000 2016,"[{'text': 'AndersonCooper', 'indices': [48, 63]}]","[{'screen_name': 'missie_88', 'name': 'Missie'...","[{'url': 'https://t.co/ayopgGMm3A', 'expanded_...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,0,2,...,False,None,22309,116010,23617,79435,195,None,False,Mon Oct 12 21:32:28 +0000 2015
3,787695338559643651,RT @WSJ: Hacked emails posted by WikiLeaks rev...,Sun Oct 16 16:42:58 +0000 2016,[],"[{'screen_name': 'WSJ', 'name': 'The Wall Stre...","[{'url': 'https://t.co/5xsXqm3kdq', 'expanded_...","<a href=""http://twitter.com/#!/download/ipad"" ...",en,0,422,...,False,None,937,169196,373,7183,194,None,False,Mon Apr 13 08:42:36 +0000 2009
4,787695338714824704,RT @postpolitics: New Washington Post-ABC News...,Sun Oct 16 16:42:58 +0000 2016,[],"[{'screen_name': 'postpolitics', 'name': 'Post...",[],"<a href=""http://twitter.com/download/android"" ...",en,0,260,...,False,None,1170,42923,1912,91489,56,None,False,Tue Dec 02 20:05:17 +0000 2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,787695377616994304,RT @isareport: Bill Clinton’s taxpayer-funded ...,Sun Oct 16 16:43:07 +0000 2016,[],"[{'screen_name': 'isareport', 'name': 'Isabel ...","[{'url': 'https://t.co/FJvWKlEjgg', 'expanded_...","<a href=""http://twitter.com/download/android"" ...",en,0,834,...,False,None,12,1031,150,4991,0,None,False,Tue Aug 30 21:23:50 +0000 2016
221,787695377604501504,RT @JLo: Let's Get Loud! #ImWithHer https://t....,Sun Oct 16 16:43:07 +0000 2016,"[{'text': 'ImWithHer', 'indices': [25, 35]}]","[{'screen_name': 'JLo', 'name': 'jlo', 'id': 8...","[{'url': 'https://t.co/7b3ID56XVC', 'expanded_...","<a href=""http://twitter.com/#!/download/ipad"" ...",en,0,708,...,False,None,3896,67434,2315,71873,36,None,False,Thu Jan 19 05:16:07 +0000 2012
222,787695378946596864,#hackers #virus Why Putin fears a Clinton win...,Sun Oct 16 16:43:07 +0000 2016,"[{'text': 'hackers', 'indices': [0, 8]}, {'tex...",[],"[{'url': 'https://t.co/FIDF8lQhW0', 'expanded_...","<a href=""http://twitterfeed.com"" rel=""nofollow...",en,0,0,...,False,None,43,18464,0,0,2,None,False,Sat May 17 17:58:39 +0000 2014
223,787695379299065856,Mike Pence: ‘We Will Absolutely Accept the Res...,Sun Oct 16 16:43:07 +0000 2016,[],[],"[{'url': 'https://t.co/meN7T55YdO', 'expanded_...","<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",en,0,0,...,False,None,38,61743,0,0,35,None,False,Mon Mar 14 09:38:18 +0000 2016


In [321]:
normal_tweets_f3_9.to_csv('normal_tweets_f3_9.csv')

In [320]:
normal_id_3[normal_id_3['tweet_id'] == 787695378875478018] 

,tweet_id
4998,787695378875478018


In [190]:
#Moving on to #4 now.
filter4 = np.loadtxt(fname='election-filter4.txt', dtype=int)
#filter3 #This is a LOT of id #s
normal_id_4 = pd.DataFrame(filter4, columns = ['tweet_id'])
normal_id_4

,tweet_id
0,777559127203524608
1,777559126985412612
2,777559127581085697
3,777559127342059520
4,777559127753056257
...,...
49999995,793158959255261185
49999996,793158979220148224
49999997,793158980348420096
49999998,793158980465864704


In [228]:
list_t_id = []
list_t_text = []
list_t_created_at = []
list_t_hashtags = []
list_t_mentions = []
list_t_urls = []
list_t_source = []
list_t_lang = []
list_t_favorite_count = []
list_t_retweet_count = []
list_t_retweeted = []
list_t_retweet_status_id = []
list_t_in_reply_to_status_id = []
    
list_user_id = []
list_user_name = []
list_user_handle = []
list_user_location = []
list_user_desc = []
list_user_verified = []
list_user_lang = []
list_user_followers_count= []
list_user_status_count = []
list_user_friend_count = []
list_user_favorites_count = []
list_user_listed_count = []
list_user_time_zone = []
list_user_protected = []
list_user_created_at = []

for id in normal_id_4[4497:5000]['tweet_id']:
    def create_url(id):
    #keyword, start_date, end_date, max_results = 10 #goes into the above

    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
        search_url = 'https://api.twitter.com/1.1/statuses/show.json?id='+str(id)
    #change params based on the endpoint you are using
        query_params = {#'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    #'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
        return (search_url, query_params)

    url = create_url(id)

#keyword, start_time, end_time, max_results
    def connect_to_endpoint(url, headers, params, next_token = None):
        params['next_token'] = next_token   #params object received from create_url function
        response = requests.request("GET", url, headers = headers, params = params)
        print("Endpoint Response Code: " + str(response.status_code))
        if response.status_code != 200:
           # raise Exception(response.status_code, response.text)
            pass
        return response.json()

    json_response = connect_to_endpoint(url[0], headers, url[1])
    if len(json_response) > 1:
        t_user= json_response['user']
        #don't need to push the above, just for below
        t_id = json_response['id']
        list_t_id.append(t_id)
    
        t_text = json_response['text']
        list_t_text.append(t_text)
    
        t_created_at = json_response['created_at']
        list_t_created_at.append(t_created_at)
    
        t_hashtags = json_response['entities']['hashtags']
        list_t_hashtags.append(t_hashtags)
    
        t_mentions = json_response['entities']['user_mentions']
        list_t_mentions.append(t_mentions)
    
        t_urls = json_response['entities']['urls']
        list_t_urls.append(t_urls)
    
        t_source = json_response['source']
        list_t_source.append(t_source)
    
        t_lang = json_response['lang']
        list_t_lang.append(t_lang)
    
        t_favorite_count = json_response['favorite_count']
        list_t_favorite_count.append(t_favorite_count)
    
        t_retweet_count = json_response['retweet_count']
        list_t_retweet_count.append(t_retweet_count)
    
        t_retweeted = json_response['retweeted']
        list_t_retweeted.append(t_retweeted)
        
        if 'retweeted_status' in json_response:
            t_retweet_status_id = json_response['retweeted_status']['id']
            list_t_retweet_status_id.append(t_retweet_status_id)
        else:
            list_t_retweet_status_id.append("None")
    
        t_in_reply_to_status_id = json_response['in_reply_to_status_id']
        list_t_in_reply_to_status_id.append(t_in_reply_to_status_id)
    
        user_id = t_user['id']
        list_user_id.append(user_id)
    
        user_name = t_user['name']
        list_user_name.append(user_name)
    
        user_handle = t_user['screen_name']
        list_user_handle.append(user_handle)
    
        user_location = t_user['location']
        list_user_location.append(user_location)
    
        user_desc = t_user['description']
        list_user_desc.append(user_desc)
    
        user_verified = t_user['verified'] 
        list_user_verified.append(user_verified)
    
        user_lang = t_user['lang']
        list_user_lang.append(user_lang)
    
        user_followers_count= t_user['followers_count']
        list_user_followers_count.append(user_followers_count)
    
        user_status_count = t_user['statuses_count']
        list_user_status_count.append(user_status_count)
    
        user_friend_count = t_user['friends_count']
        list_user_friend_count.append(user_friend_count)
    
        user_favorites_count = t_user['favourites_count']
        list_user_favorites_count.append(user_favorites_count)
    
        user_listed_count = t_user['listed_count']
        list_user_listed_count.append(user_listed_count)
    
        user_time_zone = t_user['time_zone']
        list_user_time_zone.append(user_time_zone)
    
        user_protected = t_user['protected']
        list_user_protected.append(user_protected)
    
        user_created_at = t_user['created_at']
        list_user_created_at.append(user_created_at)
    else:
        pass

Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Co

Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Co

In [229]:
completed_dict = {}
completed_dict['t_id'] = list_t_id
completed_dict['t_text'] = list_t_text
completed_dict['t_created_at'] = list_t_created_at
completed_dict['t_hashtags'] = list_t_hashtags
completed_dict['t_mentions'] = list_t_mentions
completed_dict['t_urls'] = list_t_urls
completed_dict['t_source'] = list_t_source
completed_dict['t_lang'] = list_t_lang
completed_dict['t_favorite_count'] = list_t_favorite_count
completed_dict['t_retweet_count'] = list_t_retweet_count
completed_dict['t_retweeted'] = list_t_retweeted
completed_dict['t_retweet_status_id'] = list_t_retweet_status_id
completed_dict['t_in_reply_to_status_id'] = list_t_in_reply_to_status_id
    
completed_dict['user_id'] = list_user_id
completed_dict['user_name'] = list_user_name
completed_dict['user_handle'] = list_user_handle
completed_dict['user_locatoin'] = list_user_location
completed_dict['user_desc'] = list_user_desc
completed_dict['user_verified'] = list_user_verified
completed_dict['user_lang'] = list_user_lang
completed_dict['followers_count'] = list_user_followers_count
completed_dict['user_status_count'] = list_user_status_count
completed_dict['user_friend_count'] = list_user_friend_count
completed_dict['user_favorites_count'] = list_user_favorites_count
completed_dict['user_listed_count'] = list_user_listed_count
completed_dict['user_time_zone'] = list_user_time_zone
completed_dict['user_protected'] = list_user_protected
completed_dict['user_created_at'] = list_user_created_at

normal_tweets_f4_9 = pd.DataFrame.from_dict(completed_dict)
normal_tweets_f4_9

,t_id,t_text,t_created_at,t_hashtags,t_mentions,t_urls,t_source,t_lang,t_favorite_count,t_retweet_count,...,user_verified,user_lang,followers_count,user_status_count,user_friend_count,user_favorites_count,user_listed_count,user_time_zone,user_protected,user_created_at
0,777559664091340800,RT @FirebaughNorman: Obama Is Releasing Terror...,Sun Sep 18 17:27:24 +0000 2016,[],"[{'screen_name': 'FirebaughNorman', 'name': 'N...","[{'url': 'https://t.co/34JcGGqgIe', 'expanded_...","<a href=""http://twitter.com/download/android"" ...",en,0,1,...,False,None,4508,266331,5164,268780,121,None,False,Thu Oct 11 01:18:27 +0000 2012
1,777559663793537026,that's my boy https://t.co/qPDwTkGb5x,Sun Sep 18 17:27:24 +0000 2016,[],[],"[{'url': 'https://t.co/qPDwTkGb5x', 'expanded_...","<a href=""http://twitter.com/download/iphone"" r...",en,0,0,...,False,None,449,20853,253,9381,18,None,False,Mon Jun 15 00:23:09 +0000 2015
2,777559664481406976,RT @T_FisherKing: Anyone in the hopeless mains...,Sun Sep 18 17:27:25 +0000 2016,[],"[{'screen_name': 'T_FisherKing', 'name': 'T. F...","[{'url': 'https://t.co/dF7mkpKYnu', 'expanded_...","<a href=""http://twitter.com/#!/download/ipad"" ...",en,0,40,...,False,None,527,70668,589,55316,32,None,False,Thu Nov 12 15:12:25 +0000 2009
3,777559664707768320,Social Democrats receive most votes in Berlin ...,Sun Sep 18 17:27:25 +0000 2016,[],[],"[{'url': 'https://t.co/NS48NSKOXp', 'expanded_...","<a href=""http://twitterfeed.com"" rel=""nofollow...",en,0,0,...,False,None,1453,24360,1701,0,46,None,False,Sat Jun 05 13:17:32 +0000 2010
4,777559665898950656,RT @WhatTheFFacts: From 2001 to 2009 Donald Tr...,Sun Sep 18 17:27:25 +0000 2016,[],"[{'screen_name': 'WhatTheFFacts', 'name': 'Wha...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,308,...,False,None,3402,55408,1097,35246,40,None,False,Fri Mar 13 00:37:42 +0000 2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,777559722102751232,RT @ddale8: Little Miss Flint met both Trump a...,Sun Sep 18 17:27:38 +0000 2016,[],"[{'screen_name': 'ddale8', 'name': 'Daniel Dal...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,35723,...,False,None,725,48923,101,62220,5,None,False,Sat Jan 25 17:53:54 +0000 2014
201,777559722689961984,#NowPlaying Don't Let Me Down - Remix de DJ Tr...,Sun Sep 18 17:27:38 +0000 2016,"[{'text': 'NowPlaying', 'indices': [0, 11]}]",[],"[{'url': 'https://t.co/MCkM8bugVd', 'expanded_...","<a href=""http://spotify.com"" rel=""nofollow"">Sp...",en,1,0,...,False,None,442,8869,287,7603,0,None,False,Mon Jul 23 10:26:22 +0000 2012
202,777559722777911296,RT @WhatTheFFacts: From 2001 to 2009 Donald Tr...,Sun Sep 18 17:27:38 +0000 2016,[],"[{'screen_name': 'WhatTheFFacts', 'name': 'Wha...",[],"<a href=""http://www.twitter.com"" rel=""nofollow...",en,0,308,...,False,None,700,43468,719,1871,7,None,False,Sun Nov 14 05:34:31 +0000 2010
203,777559722438242304,Pendukung Brexit Optimis Donad Trump Jadi Pres...,Sun Sep 18 17:27:38 +0000 2016,[],[],"[{'url': 'https://t.co/qPNd6RLzOy', 'expanded_...","<a href=""http://twitterfeed.com"" rel=""nofollow...",in,0,0,...,False,None,496,249572,124,9,3,None,False,Thu Feb 23 11:48:18 +0000 2012


In [231]:
normal_tweets_f4_9.to_csv('normal_tweets_f4_9.csv')

In [230]:
normal_id_4[normal_id_4['tweet_id'] == 777559722979459072] 

,tweet_id
4999,777559722979459072


In [223]:
#Moving on to #5 now.
filter5 = np.loadtxt(fname='election-filter5.txt', dtype=int)
#filter5 #This is a LOT of id #s
normal_id_5 = pd.DataFrame(filter5, columns = ['tweet_id'])
normal_id_5

,tweet_id
0,793158967052697600
1,793158965886529536
2,793158967593611264
3,793158980717547520
4,793158980616937472
...,...
49999995,789250598222852097
49999996,789250598260600832
49999997,789250598168256512
49999998,789250598529105920


In [270]:
list_t_id = []
list_t_text = []
list_t_created_at = []
list_t_hashtags = []
list_t_mentions = []
list_t_urls = []
list_t_source = []
list_t_lang = []
list_t_favorite_count = []
list_t_retweet_count = []
list_t_retweeted = []
list_t_retweet_status_id = []
list_t_in_reply_to_status_id = []
    
list_user_id = []
list_user_name = []
list_user_handle = []
list_user_location = []
list_user_desc = []
list_user_verified = []
list_user_lang = []
list_user_followers_count= []
list_user_status_count = []
list_user_friend_count = []
list_user_favorites_count = []
list_user_listed_count = []
list_user_time_zone = []
list_user_protected = []
list_user_created_at = []

for id in normal_id_5[4500:5000]['tweet_id']:
    def create_url(id):
    #keyword, start_date, end_date, max_results = 10 #goes into the above

    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
        search_url = 'https://api.twitter.com/1.1/statuses/show.json?id='+str(id)
    #change params based on the endpoint you are using
        query_params = {#'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    #'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
        return (search_url, query_params)

    url = create_url(id)

#keyword, start_time, end_time, max_results
    def connect_to_endpoint(url, headers, params, next_token = None):
        params['next_token'] = next_token   #params object received from create_url function
        response = requests.request("GET", url, headers = headers, params = params)
        print("Endpoint Response Code: " + str(response.status_code))
        if response.status_code != 200:
           # raise Exception(response.status_code, response.text)
            pass
        return response.json()

    json_response = connect_to_endpoint(url[0], headers, url[1])
    if len(json_response) > 1:
        t_user= json_response['user']
        #don't need to push the above, just for below
        t_id = json_response['id']
        list_t_id.append(t_id)
    
        t_text = json_response['text']
        list_t_text.append(t_text)
    
        t_created_at = json_response['created_at']
        list_t_created_at.append(t_created_at)
    
        t_hashtags = json_response['entities']['hashtags']
        list_t_hashtags.append(t_hashtags)
    
        t_mentions = json_response['entities']['user_mentions']
        list_t_mentions.append(t_mentions)
    
        t_urls = json_response['entities']['urls']
        list_t_urls.append(t_urls)
    
        t_source = json_response['source']
        list_t_source.append(t_source)
    
        t_lang = json_response['lang']
        list_t_lang.append(t_lang)
    
        t_favorite_count = json_response['favorite_count']
        list_t_favorite_count.append(t_favorite_count)
    
        t_retweet_count = json_response['retweet_count']
        list_t_retweet_count.append(t_retweet_count)
    
        t_retweeted = json_response['retweeted']
        list_t_retweeted.append(t_retweeted)
        
        if 'retweeted_status' in json_response:
            t_retweet_status_id = json_response['retweeted_status']['id']
            list_t_retweet_status_id.append(t_retweet_status_id)
        else:
            list_t_retweet_status_id.append("None")
    
        t_in_reply_to_status_id = json_response['in_reply_to_status_id']
        list_t_in_reply_to_status_id.append(t_in_reply_to_status_id)
    
        user_id = t_user['id']
        list_user_id.append(user_id)
    
        user_name = t_user['name']
        list_user_name.append(user_name)
    
        user_handle = t_user['screen_name']
        list_user_handle.append(user_handle)
    
        user_location = t_user['location']
        list_user_location.append(user_location)
    
        user_desc = t_user['description']
        list_user_desc.append(user_desc)
    
        user_verified = t_user['verified'] 
        list_user_verified.append(user_verified)
    
        user_lang = t_user['lang']
        list_user_lang.append(user_lang)
    
        user_followers_count= t_user['followers_count']
        list_user_followers_count.append(user_followers_count)
    
        user_status_count = t_user['statuses_count']
        list_user_status_count.append(user_status_count)
    
        user_friend_count = t_user['friends_count']
        list_user_friend_count.append(user_friend_count)
    
        user_favorites_count = t_user['favourites_count']
        list_user_favorites_count.append(user_favorites_count)
    
        user_listed_count = t_user['listed_count']
        list_user_listed_count.append(user_listed_count)
    
        user_time_zone = t_user['time_zone']
        list_user_time_zone.append(user_time_zone)
    
        user_protected = t_user['protected']
        list_user_protected.append(user_protected)
    
        user_created_at = t_user['created_at']
        list_user_created_at.append(user_created_at)
    else:
        pass

Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Co

Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Co

In [271]:
completed_dict = {}
completed_dict['t_id'] = list_t_id
completed_dict['t_text'] = list_t_text
completed_dict['t_created_at'] = list_t_created_at
completed_dict['t_hashtags'] = list_t_hashtags
completed_dict['t_mentions'] = list_t_mentions
completed_dict['t_urls'] = list_t_urls
completed_dict['t_source'] = list_t_source
completed_dict['t_lang'] = list_t_lang
completed_dict['t_favorite_count'] = list_t_favorite_count
completed_dict['t_retweet_count'] = list_t_retweet_count
completed_dict['t_retweeted'] = list_t_retweeted
completed_dict['t_retweet_status_id'] = list_t_retweet_status_id
completed_dict['t_in_reply_to_status_id'] = list_t_in_reply_to_status_id
    
completed_dict['user_id'] = list_user_id
completed_dict['user_name'] = list_user_name
completed_dict['user_handle'] = list_user_handle
completed_dict['user_locatoin'] = list_user_location
completed_dict['user_desc'] = list_user_desc
completed_dict['user_verified'] = list_user_verified
completed_dict['user_lang'] = list_user_lang
completed_dict['followers_count'] = list_user_followers_count
completed_dict['user_status_count'] = list_user_status_count
completed_dict['user_friend_count'] = list_user_friend_count
completed_dict['user_favorites_count'] = list_user_favorites_count
completed_dict['user_listed_count'] = list_user_listed_count
completed_dict['user_time_zone'] = list_user_time_zone
completed_dict['user_protected'] = list_user_protected
completed_dict['user_created_at'] = list_user_created_at

normal_tweets_f5_9 = pd.DataFrame.from_dict(completed_dict)
normal_tweets_f5_9

,t_id,t_text,t_created_at,t_hashtags,t_mentions,t_urls,t_source,t_lang,t_favorite_count,t_retweet_count,...,user_verified,user_lang,followers_count,user_status_count,user_friend_count,user_favorites_count,user_listed_count,user_time_zone,user_protected,user_created_at
0,793159345102065664,RT @jamisonfoser: Donald Trump wants to get ri...,Mon Oct 31 18:34:58 +0000 2016,[],"[{'screen_name': 'jamisonfoser', 'name': 'Jami...",[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,0,73,...,False,None,15937,133415,17491,38190,437,None,False,Fri Jul 03 05:56:30 +0000 2009
1,793159345353527296,RT @CapehartJ: Trump is the reckless human wre...,Mon Oct 31 18:34:58 +0000 2016,[],"[{'screen_name': 'CapehartJ', 'name': 'Jonatha...","[{'url': 'https://t.co/Lg8xGpjXIS', 'expanded_...","<a href=""http://twitter.com/#!/download/ipad"" ...",en,0,131,...,False,None,2400,235759,2806,205979,117,None,False,Tue Jan 19 01:53:58 +0000 2016
2,793159345043218432,RT @Hillary_esp: Esta familia latina de Arizon...,Mon Oct 31 18:34:58 +0000 2016,[],"[{'screen_name': 'Hillary_esp', 'name': 'Hilla...",[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",es,0,153,...,False,None,1981,464616,1409,174428,0,None,False,Mon Sep 06 20:39:16 +0000 2010
3,793159345462734848,RT @thehill: FBI agents pushed for Clinton Fou...,Mon Oct 31 18:34:58 +0000 2016,[],"[{'screen_name': 'thehill', 'name': 'The Hill'...","[{'url': 'https://t.co/c7u0GJLruU', 'expanded_...","<a href=""http://twitter.com/download/iphone"" r...",en,0,5102,...,False,None,165,6957,723,16267,0,None,False,Sun Dec 28 00:46:30 +0000 2008
4,793159345227857920,RT @JesseFFerguson: GOP Chairman of Judiciary ...,Mon Oct 31 18:34:58 +0000 2016,[],"[{'screen_name': 'JesseFFerguson', 'name': 'Je...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,194,...,False,None,1095,120976,926,88397,8,None,False,Sun Jun 19 00:49:25 +0000 2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,793159383253266432,RT @RawStory: Trump companies destroyed emails...,Mon Oct 31 18:35:07 +0000 2016,[],"[{'screen_name': 'RawStory', 'name': 'Raw Stor...","[{'url': 'https://t.co/0MDFuwq2sR', 'expanded_...","<a href=""http://www.twitter.com"" rel=""nofollow...",en,0,224,...,False,None,240,12946,1990,10242,11,None,False,Sat Jan 02 20:46:09 +0000 2010
227,793159383253340161,Trump ally busted for making up story about ‘s...,Mon Oct 31 18:35:07 +0000 2016,"[{'text': 'TeamHillary', 'indices': [86, 98]},...",[],"[{'url': 'https://t.co/TofoeqwVe0', 'expanded_...","<a href=""http://www.hillaryvsdonald.org"" rel=""...",en,0,1,...,False,None,22,34957,3,0,0,None,False,Sun Jul 24 06:57:11 +0000 2016
228,793159383609774081,RT @thehill: Poll: Public overwhelmingly think...,Mon Oct 31 18:35:07 +0000 2016,[],"[{'screen_name': 'thehill', 'name': 'The Hill'...","[{'url': 'https://t.co/3HnT0yROxP', 'expanded_...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,0,71,...,False,None,2488,54272,2504,97026,43,None,False,Sun Jun 08 23:24:46 +0000 2014
229,793159383593222144,WAKE ME UP https://t.co/zAJfKxb7Ub,Mon Oct 31 18:35:07 +0000 2016,[],[],"[{'url': 'https://t.co/zAJfKxb7Ub', 'expanded_...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",en,1,0,...,False,None,640,34460,629,50631,4,None,False,Sun Jul 03 18:32:59 +0000 2016


In [273]:
normal_tweets_f5_9.to_csv('normal_tweets_f5_9.csv')

In [272]:
normal_id_5[normal_id_5['tweet_id'] == 793159384079540225] 

,tweet_id
4999,793159384079540225


In [257]:
#Moving on to #6 now.
filter6 = np.loadtxt(fname='election-filter6.txt', dtype=int)
#filter6 #This is a LOT of id #s
normal_id_6 = pd.DataFrame(filter6, columns = ['tweet_id'])
normal_id_6

,tweet_id
0,789250598268837889
1,789250597526446080
2,789250598239670273
3,789250597266546688
4,789250597841084417
...,...
1376584,789425344386207744
1376585,789425345174773760
1376586,789425345661399044
1376587,789425345443291136


In [311]:
list_t_id = []
list_t_text = []
list_t_created_at = []
list_t_hashtags = []
list_t_mentions = []
list_t_urls = []
list_t_source = []
list_t_lang = []
list_t_favorite_count = []
list_t_retweet_count = []
list_t_retweeted = []
list_t_retweet_status_id = []
list_t_in_reply_to_status_id = []
    
list_user_id = []
list_user_name = []
list_user_handle = []
list_user_location = []
list_user_desc = []
list_user_verified = []
list_user_lang = []
list_user_followers_count= []
list_user_status_count = []
list_user_friend_count = []
list_user_favorites_count = []
list_user_listed_count = []
list_user_time_zone = []
list_user_protected = []
list_user_created_at = []

for id in normal_id_6[4498:5000]['tweet_id']:
    def create_url(id):
    #keyword, start_date, end_date, max_results = 10 #goes into the above

    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
        search_url = 'https://api.twitter.com/1.1/statuses/show.json?id='+str(id)
    #change params based on the endpoint you are using
        query_params = {#'query': keyword,
                    #'start_time': start_date,
                    #'end_time': end_date,
                    #'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
        return (search_url, query_params)

    url = create_url(id)

#keyword, start_time, end_time, max_results
    def connect_to_endpoint(url, headers, params, next_token = None):
        params['next_token'] = next_token   #params object received from create_url function
        response = requests.request("GET", url, headers = headers, params = params)
        print("Endpoint Response Code: " + str(response.status_code))
        if response.status_code != 200:
           # raise Exception(response.status_code, response.text)
            pass
        return response.json()

    json_response = connect_to_endpoint(url[0], headers, url[1])
    if len(json_response) > 1:
        t_user= json_response['user']
        #don't need to push the above, just for below
        t_id = json_response['id']
        list_t_id.append(t_id)
    
        t_text = json_response['text']
        list_t_text.append(t_text)
    
        t_created_at = json_response['created_at']
        list_t_created_at.append(t_created_at)
    
        t_hashtags = json_response['entities']['hashtags']
        list_t_hashtags.append(t_hashtags)
    
        t_mentions = json_response['entities']['user_mentions']
        list_t_mentions.append(t_mentions)
    
        t_urls = json_response['entities']['urls']
        list_t_urls.append(t_urls)
    
        t_source = json_response['source']
        list_t_source.append(t_source)
    
        t_lang = json_response['lang']
        list_t_lang.append(t_lang)
    
        t_favorite_count = json_response['favorite_count']
        list_t_favorite_count.append(t_favorite_count)
    
        t_retweet_count = json_response['retweet_count']
        list_t_retweet_count.append(t_retweet_count)
    
        t_retweeted = json_response['retweeted']
        list_t_retweeted.append(t_retweeted)
        
        if 'retweeted_status' in json_response:
            t_retweet_status_id = json_response['retweeted_status']['id']
            list_t_retweet_status_id.append(t_retweet_status_id)
        else:
            list_t_retweet_status_id.append("None")
    
        t_in_reply_to_status_id = json_response['in_reply_to_status_id']
        list_t_in_reply_to_status_id.append(t_in_reply_to_status_id)
    
        user_id = t_user['id']
        list_user_id.append(user_id)
    
        user_name = t_user['name']
        list_user_name.append(user_name)
    
        user_handle = t_user['screen_name']
        list_user_handle.append(user_handle)
    
        user_location = t_user['location']
        list_user_location.append(user_location)
    
        user_desc = t_user['description']
        list_user_desc.append(user_desc)
    
        user_verified = t_user['verified'] 
        list_user_verified.append(user_verified)
    
        user_lang = t_user['lang']
        list_user_lang.append(user_lang)
    
        user_followers_count= t_user['followers_count']
        list_user_followers_count.append(user_followers_count)
    
        user_status_count = t_user['statuses_count']
        list_user_status_count.append(user_status_count)
    
        user_friend_count = t_user['friends_count']
        list_user_friend_count.append(user_friend_count)
    
        user_favorites_count = t_user['favourites_count']
        list_user_favorites_count.append(user_favorites_count)
    
        user_listed_count = t_user['listed_count']
        list_user_listed_count.append(user_listed_count)
    
        user_time_zone = t_user['time_zone']
        list_user_time_zone.append(user_time_zone)
    
        user_protected = t_user['protected']
        list_user_protected.append(user_protected)
    
        user_created_at = t_user['created_at']
        list_user_created_at.append(user_created_at)
    else:
        pass

Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Co

Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 403
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 404
Endpoint Response Code: 404
Endpoint Response Code: 403
Endpoint Response Code: 200
Endpoint Response Code: 200
Endpoint Response Co

In [312]:
completed_dict = {}
completed_dict['t_id'] = list_t_id
completed_dict['t_text'] = list_t_text
completed_dict['t_created_at'] = list_t_created_at
completed_dict['t_hashtags'] = list_t_hashtags
completed_dict['t_mentions'] = list_t_mentions
completed_dict['t_urls'] = list_t_urls
completed_dict['t_source'] = list_t_source
completed_dict['t_lang'] = list_t_lang
completed_dict['t_favorite_count'] = list_t_favorite_count
completed_dict['t_retweet_count'] = list_t_retweet_count
completed_dict['t_retweeted'] = list_t_retweeted
completed_dict['t_retweet_status_id'] = list_t_retweet_status_id
completed_dict['t_in_reply_to_status_id'] = list_t_in_reply_to_status_id
    
completed_dict['user_id'] = list_user_id
completed_dict['user_name'] = list_user_name
completed_dict['user_handle'] = list_user_handle
completed_dict['user_locatoin'] = list_user_location
completed_dict['user_desc'] = list_user_desc
completed_dict['user_verified'] = list_user_verified
completed_dict['user_lang'] = list_user_lang
completed_dict['followers_count'] = list_user_followers_count
completed_dict['user_status_count'] = list_user_status_count
completed_dict['user_friend_count'] = list_user_friend_count
completed_dict['user_favorites_count'] = list_user_favorites_count
completed_dict['user_listed_count'] = list_user_listed_count
completed_dict['user_time_zone'] = list_user_time_zone
completed_dict['user_protected'] = list_user_protected
completed_dict['user_created_at'] = list_user_created_at

normal_tweets_f6_9 = pd.DataFrame.from_dict(completed_dict)
normal_tweets_f6_9

,t_id,t_text,t_created_at,t_hashtags,t_mentions,t_urls,t_source,t_lang,t_favorite_count,t_retweet_count,...,user_verified,user_lang,followers_count,user_status_count,user_friend_count,user_favorites_count,user_listed_count,user_time_zone,user_protected,user_created_at
0,789250967950589953,"RT @HillaryClinton: This #SpiritDay, let's tak...",Thu Oct 20 23:44:28 +0000 2016,"[{'text': 'SpiritDay', 'indices': [25, 35]}]","[{'screen_name': 'HillaryClinton', 'name': 'Hi...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,7563,...,False,None,83,1795,404,9701,3,None,False,Mon May 30 05:51:36 +0000 2016
1,789250967879372800,RT @HillaryClinton: Our democracy is built on ...,Thu Oct 20 23:44:28 +0000 2016,[],"[{'screen_name': 'HillaryClinton', 'name': 'Hi...",[],"<a href=""http://twitter.com/download/android"" ...",en,0,5535,...,False,None,1412,13086,889,25981,80,None,False,Tue Nov 04 00:24:06 +0000 2014
2,789250968068038656,RT @2nd_To_None: SHE IS A SECURITY RISK WE CAN...,Thu Oct 20 23:44:29 +0000 2016,[],"[{'screen_name': '2nd_To_None', 'name': '2nd_T...",[],"<a href=""http://twitter.com/download/android"" ...",en,0,12,...,False,None,2991,141768,3369,388664,31,None,False,Wed Mar 21 22:47:52 +0000 2012
3,789250968193945600,Acaso eres un #badhombres @pachecotv ? https:/...,Thu Oct 20 23:44:29 +0000 2016,"[{'text': 'badhombres', 'indices': [14, 25]}]","[{'screen_name': 'pachecotv', 'name': 'pacheco...","[{'url': 'https://t.co/XmmUpzhbfN', 'expanded_...","<a href=""http://twitter.com/#!/download/ipad"" ...",es,0,0,...,False,None,930,43786,1302,1672,38,None,False,Wed May 28 16:38:11 +0000 2008
4,789250969292840961,"RT @hunter14tigers: Trump or Hilary, neither w...",Thu Oct 20 23:44:29 +0000 2016,[],"[{'screen_name': 'hunter14tigers', 'name': 'Hu...",[],"<a href=""http://twitter.com/download/iphone"" r...",en,0,19,...,False,None,284,600,298,1017,0,None,False,Sun Mar 30 03:32:34 +0000 2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,789251007490301952,"RT @VICE: Trump's ""nasty woman"" comment is now...",Thu Oct 20 23:44:38 +0000 2016,[],"[{'screen_name': 'VICE', 'name': 'VICE', 'id':...","[{'url': 'https://t.co/5Y4uG75s11', 'expanded_...","<a href=""http://www.echofon.com/"" rel=""nofollo...",en,0,210,...,False,None,5366,41457,727,1947,25,None,False,Wed Jul 07 09:15:36 +0000 2010
204,789251009809899521,RT @HillaryClinton: Trump wouldn't say if he'l...,Thu Oct 20 23:44:38 +0000 2016,[],"[{'screen_name': 'HillaryClinton', 'name': 'Hi...",[],"<a href=""http://twitter.com/download/android"" ...",en,0,1451,...,False,None,10280,103394,8172,104783,123,None,False,Wed Oct 07 20:41:49 +0000 2015
205,789251009491111936,@TylerBoliver @NoahCRothman Isn't it odd how t...,Thu Oct 20 23:44:38 +0000 2016,[],"[{'screen_name': 'TylerBoliver', 'name': 'Tyle...",[],"<a href=""http://twitter.com/#!/download/ipad"" ...",en,3,0,...,False,None,2022,33466,3016,61351,17,None,False,Wed Aug 26 14:41:04 +0000 2015
206,789251009973329920,"RT @voxdotcom: At a rally on Thursday, Donald ...",Thu Oct 20 23:44:39 +0000 2016,[],"[{'screen_name': 'voxdotcom', 'name': 'Vox', '...",[],"<a href=""http://twitter.com/#!/download/ipad"" ...",en,0,85,...,False,None,1049,7924,1842,50,44,None,False,Fri Apr 08 12:58:44 +0000 2011


In [314]:
normal_tweets_f6_9.to_csv('normal_tweets_f6_9.csv')

In [313]:
normal_id_6[normal_id_6['tweet_id'] == 789251010002771968] 

,tweet_id
4999,789251010002771968
